In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

In [12]:
df = pd.read_csv('AtlantaPrices_Processed.csv')

C:\Users\morga\AppData\Local\Temp\ipykernel_21896\725447062.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('AtlantaPrices_Processed.csv')


In [13]:
df.head()

,searchDate,flightDate,destinationAirport,fareBasisCode,elapsedDays,isBasicEconomy,isNonStop,baseFare,totalFare,seatsRemaining,...,segmentsDurationInSeconds_4,segmentsDistance_1,segmentsDistance_2,segmentsDistance_3,segmentsDistance_4,segmentsCabinCode_1,segmentsCabinCode_2,segmentsCabinCode_3,segmentsCabinCode_4,Unnamed: 66
0,2022-04-16,2022-04-17,BOS,LA0NX0MC,0,False,True,217.67,248.6,9,...,0,947,0,0,0,coach,0,0,0,NaN
1,2022-04-16,2022-04-17,BOS,LA0NX0MC,0,False,True,217.67,248.6,4,...,0,947,0,0,0,coach,0,0,0,NaN
2,2022-04-16,2022-04-17,BOS,LA0NX0MC,0,False,True,217.67,248.6,9,...,0,947,0,0,0,coach,0,0,0,NaN
3,2022-04-16,2022-04-17,BOS,LA0NX0MC,0,False,True,217.67,248.6,8,...,0,947,0,0,0,coach,0,0,0,NaN
4,2022-04-16,2022-04-17,BOS,LA0NX0MC,0,False,True,217.67,248.6,9,...,0,947,0,0,0,coach,0,0,0,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16003 entries, 0 to 16002
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   legId                              16003 non-null  object 
 1   searchDate                         16003 non-null  object 
 2   flightDate                         16003 non-null  object 
 3   startingAirport                    16003 non-null  object 
 4   destinationAirport                 16003 non-null  object 
 5   fareBasisCode                      16003 non-null  object 
 6   travelDuration                     16003 non-null  object 
 7   elapsedDays                        16003 non-null  float64
 8   isBasicEconomy                     16003 non-null  bool   
 9   isRefundable                       16003 non-null  bool   
 10  isNonStop                          16003 non-null  bool   
 11  baseFare                           16003 non-null  flo

In [ ]:
df['segmentsDepartureTimeRaw_1'] = pd.to_datetime(df['segmentsDepartureTimeRaw_1'], format='%Y-%m-%dT%H:%M:%S.%f%z')
df['weeknum'] = df['segmentsDepartureTimeRaw_1'].dt.isocalendar().week

columns_to_count = ['segmentsAirlineName_1', 'segmentsAirlineName_2', 'segmentsAirlineName_3', 'segmentsAirlineName_4']
df['no_airlines'] = df[columns_to_count].apply(lambda row: len(set(filter(lambda x: x != 0, row)))-1, axis=1)

columns_to_drop = ['searchDate', 'flightDate', 'fareBasisCode', 'elapsed days' 'totalFare', 'segmentsDepartureTimeEpochSeconds_1',
                   'segmentsDepartureTimeEpochSeconds_2', 'segmentsDepartureTimeEpochSeconds_3', 'segmentsDepartureTimeEpochSeconds_4',
                  'segmentsDepartureTimeRaw_2', 'segmentsDepartureTimeRaw_3','segmentsDepartureTimeRaw_4', 'segmentsArrivalTimeEpochSeconds_1',
                  'segmentsArrivalTimeEpochSeconds_2', 'segmentsArrivalTimeEpochSeconds_3', 'segmentsArrivalTimeEpochSeconds_4', 'segmentsArrivalTimeRaw_1',
                  'segmentsArrivalTimeRaw_2', 'segmentsArrivalTimeRaw_3', 'segmentsArrivalTimeRaw_4', 'segmentsArrivalAirportCode_1',
                  'segmentsArrivalAirportCode_1', 'segmentsArrivalAirportCode_2', 'segmentsArrivalAirportCode_3', 'segmentsArrivalAirportCode_4',
                  'segmentsDepartureAirportCode_1', 'segmentsDepartureAirportCode_2', 'segmentsDepartureAirportCode_3', 'segmentsDepartureAirportCode_4',
                  'segmentsAirlineName_1', 'segmentsAirlineName_2', 'segmentsAirlineName_3', 'segmentsAirlineName_4', 'segmentsAirlineCode_1',
                  'segmentsAirlineCode_2', 'segmentsAirlineCode_3', 'segmentsAirlineCode_4', 'segmentsDurationInSeconds_1', 'segmentsDurationInSeconds_2',
                  'segmentsDurationInSeconds_3', 'segmentsDurationInSeconds_4', 'segmentsDistance_1', 'segmentsDistance_2', 'segmentsDistance_3',
                  'segmentsDistance_4', 'segmentsCabinCode_2', 'segmentsCabinCode_3', 'segmentsCabinCode_4' ]

df = df.drop(columns=column_to_drop)

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA


# Step 1: One-hot encode non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=['int', 'float']).columns

one_hot_encoder = OneHotEncoder(sparse=False, drop='first')  # 'drop' is set to 'first' to avoid the dummy variable trap
encoded_columns = pd.DataFrame(one_hot_encoder.fit_transform(df[non_numeric_columns]))
encoded_columns.columns = one_hot_encoder.get_feature_names_out(non_numeric_columns)

df = pd.concat([df.drop(columns=non_numeric_columns), encoded_columns], axis=1)

# Step 2: Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

# Step 3: Perform PCA
# You can specify the number of components you want to keep
num_components = 10  # Change this to your desired number of components
pca = PCA(n_components=num_components)
pca_result = pca.fit_transform(scaled_data)

# Now, pca_result contains the transformed data after PCA
# You can convert it back to a DataFrame if needed
pca_dataframe = pd.DataFrame(data=pca_result, columns=[f'PC{i+1}' for i in range(num_components)])

# Print the resulting DataFrame
print(pca_dataframe.head())


In [ ]:
#Perform PCA with automatic determination of components
pca = PCA()
pca_result = pca.fit_transform(scaled_data)

# Determine the number of components based on a threshold (e.g., 95% variance explained)
cumulative_variance_ratio = pca.explained_variance_ratio_.cumsum()
num_components = sum(cumulative_variance_ratio >= 0.95)

# Apply PCA with the determined number of components
pca = PCA(n_components=num_components)
pca_result = pca.fit_transform(scaled_data)

# Now, pca_result contains the transformed data after PCA
pca_dataframe = pd.DataFrame(data=pca_result, columns=[f'PC{i+1}' for i in range(num_components)])

# Print the resulting DataFrame and explained variance
print(pca_dataframe.head())
print(f"Number of components: {num_components}")
print(f"Cumulative explained variance: {cumulative_variance_ratio[num_components - 1]:.2%}")
